In [1]:
#%%
import os
data_dir=os.path.join(os.getcwd(), "../../DataSets/DogsVsCats/")
print(os.listdir(data_dir))
src_train=os.path.join(data_dir, "train")
src_test=os.path.join(data_dir, "test1")

src_train_images=os.listdir(src_train)
src_test_images=os.listdir(src_test)

['.DS_Store', 'test1', 'test1.zip', 'train', 'train.zip']


In [2]:
import os, shutil
original_dataset_dir = os.path.join(os.getcwd(), "../../DataSets/DogsVsCats/")
src_train=os.path.join(original_dataset_dir, "train")
src_test=os.path.join(original_dataset_dir, "test1")

base_dir = os.path.join(os.getcwd(),"../data/cats_and_dogs_small")
#os.mkdir(base_dir)
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)
train_cats_dir = os.path.join(train_dir, 'cats')
#os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
#os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
#os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
#os.mkdir(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
#os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
#os.mkdir(test_dogs_dir)


In [3]:
#%% keras image generators
from keras_preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(1./255)
test_datagen=ImageDataGenerator(1./255)

train_generator=train_datagen.flow_from_directory(directory=train_dir, target_size=(150,150), color_mode='rgb', class_mode='binary', batch_size=20)

validation_generator=test_datagen.flow_from_directory(directory=validation_dir, target_size=(150,150), color_mode='rgb', class_mode='binary', batch_size=20)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
#%% init model
from keras import models, layers
model=models.Sequential()
model.add(layers.Conv2D(32, (3,3), input_shape=(150,150,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=1, activation='sigmoid'))

model.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)      

In [5]:
#%% compile model
from keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=1e-4), loss='binary_crossentropy', metrics=['acc'])


In [6]:
#%% fit model
history=model.fit_generator(train_generator, epochs=2, steps_per_epoch=100, validation_data=validation_generator, validation_steps=50)


Epoch 1/2


/Users/unravel/anaconda3/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


100/100 [==============================] - 41s 405ms/step - loss: 4.5501 - acc: 0.5315 - val_loss: 0.5100 - val_acc: 0.5830
Epoch 2/2
100/100 [==============================] - 42s 421ms/step - loss: 0.7317 - acc: 0.6025 - val_loss: 0.6858 - val_acc: 0.5800


In [ ]:
#%% plot history
import matplotlib.pyplot as plt

hist_dict=history.history
acc=hist_dict['acc']
loss=hist_dict['loss']
val_acc=hist_dict['val_acc']
val_loss=hist_dict['val_loss']
epochs=range(1, len(acc)+1)
plt.plot(epochs,acc,'bo', label='Training acc')
plt.plot(epochs,val_acc,'ro', label='Validation acc')
plt.legend()
plt.figure()
plt.plot(epochs,loss,'b', label='Training loss')
plt.plot(epochs,val_loss,'r', label='Validation loss')
plt.legend()
#plt.figure()
plt.show()